# Adaptive PDE discretizations on cartesian grids 
## Volume : GPU accelerated methods
## Part : Reproducibility
## Chapter : Isotropic metrics

In this notebook, we solve isotropic eikonal equations on the CPU and the GPU, and check that they produce consistent results.

We obtain substantial accelerations on sufficiently large instances, a few million points, by a factor up to $100$. Note that smaller test cases yield less acceleration, due to the difficulty to extract parallelism.

**bit-consistency** By design, the CPU and GPU codes produce the same values, up to machine precision, which is approximately $10^{-8}$ for floating point types, in a variety on situations. 
This is due to the fact that the two implementations, although widely different, solve the same discretized problem. We try to ensure this behavior and check it as much as possible, as it is a strong indicator of the validity of the implementations. We refer to it as bit-consistency, aknowledging that it is a bit abusive, since the algorithm outputs, while extremely close, are still far from a bit-for-bit match.

[**Summary**](Summary.ipynb) of volume GPU accelerated methods, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1 Three dimensions](#1-Three-dimensions)
  * [2 Two dimensions](#2-Two-dimensions)
  * [3. GPU specific options](#3.-GPU-specific-options)
    * [3.1 Multiprecision](#3.1-Multiprecision)
    * [3.2 Block shape](#3.2-Block-shape)
    * [3.3 Periodicity](#3.3-Periodicity)
    * [3.4 Help and parameter defaults](#3.4-Help-and-parameter-defaults)



**Acknowledgement.** The experiments presented in these notebooks are part of ongoing research.
The author would like to acknowledge fruitful informal discussions with L. Gayraud on the 
topic of GPU coding and optimization.

Copyright Jean-Marie Mirebeau, University Paris-Sud, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Isotropic_Repro','GPU'))

In [2]:
import cupy as cp
import numpy as np
import itertools
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

In [3]:
from agd import HFMUtils
from agd import AutomaticDifferentiation as ad
from agd import Metrics
import agd.AutomaticDifferentiation.cupy_generic as cugen
norm_infinity = ad.Optimization.norm_infinity
from agd.HFMUtils import RunGPU,RunSmart

In [4]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global HFMUtils,ad,cugen,RunGPU
    HFMUtils,ad,cugen,RunGPU = rreload([HFMUtils,ad,cugen,RunGPU],"../..")    

### 0.1 Decorations for gpu usage

Dealing with GPU data induces minor inconveniences:
- GPU arrays are not implicitly convertible to CPU arrays, and this is a good thing since memory transfers from GPU memory to CPU memory, and conversely, are not cheap. The `get` method must be applied to a cupy array to retrieve a numpy array.
- GPU computing is much more efficient with 32 bit data types, integer and floating point, than with their 64bit counterparts. However a number of numpy and cupy basic functions default to 64bit output, which will be inconsistent with the rest of computations.

We provide decorators to perform these memory transfers and data type conversions automatically. They are only applied to specific modules and functions, below, to avoid excessive implicit operations. 

In [5]:
cp = cugen.decorate_module_functions(cp,cugen.set_output_dtype32) # Use float32 and int32 types in place of float64 and int64
plt = cugen.decorate_module_functions(plt,cugen.cupy_get_args)
RunSmart = cugen.cupy_get_args(RunSmart,dtype64=True,iterables=(dict,Metrics.Base))

AttributeError: module 'agd.AutomaticDifferentiation.cupy_generic' has no attribute 'decorate_module_functions'

In [ ]:
X = cp.linspace(0,np.pi) # 64 bit output is converted to 32 bit by decorator
plt.plot(X,np.sin(X));   # GPU array is transfered to CPU memory.

Note that, by default, these decorators do not apply to the system module, by only to a shallow copy.

In [ ]:
sys.modules['cupy'].linspace(0,np.pi).dtype

### 0.2 Utility functions

In [ ]:
def RunCompare(gpuIn,check=True):
    gpuOut = RunGPU(gpuIn)
    if gpuIn.get('verbosity',1): print("---")
    cpuOut = RunSmart(gpuIn)
    print("Max |gpuValues-cpuValues| : ", norm_infinity(gpuOut['values'].get()-cpuOut['values']))
    cpuTime = cpuOut['FMCPUTime']; gpuTime = gpuOut['solverGPUTime'];
    print(f"Solver time (s). GPU : {gpuTime}, CPU : {cpuTime}. Device acceleration : {cpuTime/gpuTime}")
    assert not check or cp.allclose(gpuOut['values'],cpuOut['values'],atol=1e-6)
    return gpuOut,cpuOut

## 1 Three dimensions

GPU acceleration shines particularly well in three dimensions, where we get accelerations by a factor $100$ on common sizes.

In [ ]:
n = 200 # Typical instance size
hfmIn = HFMUtils.dictIn({
    'model':'Isotropic3',
    'seeds':[[0.,0.5,1.]],
    'exportValues':1,
    'cost':cp.array(1.),
})
hfmIn.SetRect([[0,1],[0,1],[0,1]],dimx=n+1,sampleBoundary=True)

In [ ]:
RunCompare(hfmIn);

We check bit-consistency for a few variants of the scheme.

In [ ]:
n=50; hfmInS = hfmIn.copy() # Define the a smaller instance
hfmInS.SetRect([[0,1],[0,1],[0,1]],dimx=n+1,sampleBoundary=True)
X = hfmInS.Grid()
hfmInS.update({
    'cost': np.prod(np.sin(2*np.pi*X)) + 1.1, # Non-constant cost
    'verbosity':0,
})

In [ ]:
factor_variants = [
    {}, # Default
    {"seedRadius":2}, # Spread seed information
    {"factorizationRadius":10,'factorizationPointChoice':'Key'} # Source factorization
]
multip_variants = [
    {'multiprecision':False}, # Default
    {'multiprecision':True}, # Reduces roundoff errors
]
order_variants = [
    {'order':1}, # Default
    {'order':2}, # More accurate on smooth instances
]

In [ ]:
for fact,multip in itertools.product(factor_variants,multip_variants):
    print(f"\nReproducibility with options : {fact}, {multip}")
    RunCompare({**hfmInS,**fact,**multip})

The second order scheme implementation has some slight differences between the GPU and CPU implementation, hence *one cannot expect bit-consistency in general*. Still, it may happen sometimes, especially when the seed is in a corner.

In [ ]:
hfmInS.update({
    'seeds':[[0.0,0.,1.]],
    'order':2,
})

In [ ]:
for fact,multip in itertools.product((factor_variants[0],factor_variants[2]),multip_variants):
    print(f"\nReproducibility with options : {fact}, {multip}")
    RunCompare({**hfmInS,**fact,**multip})

## 2 Two dimensions

In two dimensions, it is usually more difficult to extract parallism than in three dimensions.
Indeed, the front propagated in the computations is expected to have approximately
$$
    N^{\frac{d-1} d}
$$
points in dimension $d$, where $N$ denotes the total number of points in the domain. 
The front in a two dimensional computation thus has $N^{\frac 1 2}$ points which is much fewer than $N^{\frac 2 3}$ in three dimensions, when $N$ is large. In addition the number of points $N$ is also often fewer in two dimensional problems.

In [ ]:
n=4000
hfmIn = HFMUtils.dictIn({
    'model':'Isotropic2',
    'seeds':[[0.,0.5]],
    'exportValues':1,
    'cost':cp.array(1.),
})
hfmIn.SetRect([[0,1],[0,1]],dimx=n+1,sampleBoundary=True)

In [ ]:
_,cpuOut = RunCompare(hfmIn,check=False)

Another annoyance is that the numerical error, close to $10^{-4}$, is not as good as could be expected, around $10^{-7}$ for single precision floating point types. 
A quick fix, explained in more detail below and which does have a computational cost, is to run the computation using multiprecision.

In [ ]:
gpuOut = RunGPU({**hfmIn,'multiprecision':True})
print("Max |gpuValues-cpuValues| : ", norm_infinity(gpuOut['values'].get()-cpuOut['values']))

In [ ]:
n=200; hfmInS = hfmIn.copy() # Define a small instance for bit-consistency validation
hfmInS.SetRect([[0,1],[0,1]],dimx=n+1,sampleBoundary=True)
X = hfmInS.Grid()
hfmInS.update({
    'cost':np.prod(np.sin(2*np.pi*X)) +1.1, # Non-constant cost
    'verbosity':0,
})

In [ ]:
for fact,multip in itertools.product(factor_variants,multip_variants):
    print(f"\nReproducibility with options : {fact}, {multip}")
    RunCompare({**hfmInS,**fact,**multip})

Again, bit consistency is not expected with the second order scheme, but not excluded either.

In [ ]:
hfmInS.update({
    'seeds':[[0.0,1.]],
    'order':2,
})

In [ ]:
for fact,multip in itertools.product((factor_variants[0],factor_variants[2]),multip_variants):
    print(f"\nReproducibility with options : {fact}, {multip}")
    RunCompare({**hfmInS,**fact,**multip})

## 3. GPU specific options



### 3.1 Multiprecision

In multi-precision mode, the front values are represented as pairs 
$$
    u(x) = u_q(x) \delta + u_r(x),
$$
where $u_q(x) \in Z$ is an integer, $\delta>0$ is a fixed scale, and $u_r(x) \in [-\delta/2,\delta/2[$.

The parameter $\delta$ is set automatically as the largest power of two (usually a negative power) bounded by $h/10$, where $h$ is the grid scale. The choice of a power of two avoids roundoff errors.

Eventually, the result is converted to floating point format. A slightly better accuracy can be obtained by using a double type in this last step.

In [ ]:
gpuOut = RunGPU({**hfmIn,'values_float64':True})
print("Max |gpuValues-cpuValues| : ", norm_infinity(gpuOut['values'].get()-cpuOut['values']))

### 3.2 Block shape

The GPU implementation of the fast marching method works by grouping together blocks of grid points, which are updated simultaneously a prescribed number of times. The shape of these blocks ` shape_i`  and number of iterations `niter_i` can be modified, which may affect performance. They are collected in the `traits` input parameter, together with a number of compile time constants and typedefs for the GPU kernel.

In [ ]:
gpuOut['keys']['defaulted']['traits']

### 3.3 Periodicity

Standard periodic boundary conditions may be applied to one or several some axes.

In [ ]:
n=48
hfmIn = HFMUtils.dictIn({
    'model':'Isotropic2',
    'seeds':[[0.2,0.4]],
    'exportValues':1,
    'cost':cp.array(1.),
    'verbosity':1,
    'periodic':(False,True) # Periodic along second axis
})
hfmIn.SetRect([[0,1],[0,1]],dimx=n+1,sampleBoundary=False)

In [ ]:
gpuOut = RunGPU(hfmIn)

In [ ]:
X = hfmIn.Grid()
plt.title('Periodic solution'); plt.axis('equal')
plt.contour(*X,gpuOut['values']); 

### 3.4 Help and parameter defaults

The HFM algorithm sets a number of parameters to (hopefull relevant) default values. 
Setting a verbosity equal or larger than two will display some of the associated values.

In [ ]:
n=200
hfmIn = HFMUtils.dictIn({
    'model':'Isotropic2',
    'seeds':[[0.,0.5]],
    'exportValues':1,
    'cost':cp.array(1.),
    'verbosity':2
})
hfmIn.SetRect([[0,1],[0,1]],dimx=n+1,sampleBoundary=True)

In [ ]:
gpuOut = RunGPU(hfmIn)

The defaulted keys and values are also presented in the output.

In [ ]:
gpuOut['keys']['defaulted']

A succint help on the role of each key can be displayed.

In [ ]:
hfmIn['help'] = [key for key,_ in gpuOut['keys']['defaulted']] # Display help on these keys
hfmIn['verbosity']=1

In [ ]:
gpuOut = RunGPU(hfmIn)